<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Upper Type Bounds

In Scala, [type variables](generic-classes.ipynb) and [abstract types](abstract-types.ipynb) may be constrained by a type bound. Such type bounds limit the concrete values of the type variables and possibly reveal more information about the members of such types. 

An _upper type bound_ or _type constraint_ `T <: A` declares that type variable `T` refers to a subtype of type `A`.
Here is an example that demonstrates upper type bound for a type parameter of class `PetContainer`.

In [1]:
abstract class Animal {
 def name: String
}

abstract class Pet extends Animal {}

class Cat extends Pet {
  override def name: String = "Cat"
}

class Dog extends Pet {
  override def name: String = "Dog"
}

class Lion extends Animal {
  override def name: String = "Lion"
}

class PetContainer[P <: Pet](p: P) {
  def pet: P = p
}

val dogContainer = new PetContainer[Dog](new Dog)
val catContainer = new PetContainer[Cat](new Cat)

defined class Animal
defined class Pet
defined class Cat
defined class Dog
defined class Lion
defined class PetContainer
dogContainer: PetContainer[Dog] = ammonite.$sess.cmd0$Helper$PetContainer@5ce7ef5d
catContainer: PetContainer[Cat] = ammonite.$sess.cmd0$Helper$PetContainer@5caf67b8

In [2]:
// this would not compile
val lionContainer = new PetContainer[Lion](new Lion)

cmd1.sc:104: type arguments [cmd1.this.cmd0.Lion] do not conform to class PetContainer's type parameter bounds [P <: Helper.this.Pet]
val lionContainer = new PetContainer[Lion](new Lion)
    ^cmd1.sc:1: type arguments [cmd1.this.cmd0.Lion] do not conform to class PetContainer's type parameter bounds [P <: Helper.this.Pet]
val lionContainer = new PetContainer[Lion](new Lion)
                        ^cmd1.sc:8: type arguments [cmd1.this.cmd0.Lion] do not conform to class PetContainer's type parameter bounds [P <: Helper.this.Pet]
            .print(lionContainer, "lionContainer", _root_.scala.None)
             ^cmd1.sc:8: type arguments [cmd1.this.cmd0.Lion] do not conform to class PetContainer's type parameter bounds [P <: Helper.this.Pet]
            .print(lionContainer, "lionContainer", _root_.scala.None)
                  ^Compilation Failed

: 

The `class PetContainer` take a type parameter `P` which must be a subtype of `Pet`. `Dog` and `Cat` are subtypes of `Pet` so we can create a new `PetContainer[Dog]` and `PetContainer[Cat]`. However, if we tried to create a `PetContainer[Lion]`, we would get the following Error:

`type arguments [Lion] do not conform to class PetContainer's type parameter bounds [P <: Pet]`

This is because `Lion` is not a subtype of `Pet`.

## Another example

There are types and methods that, even if they work for a wide variety of types, 
they do not work for all. For example, the type `Set[T]`, where `T` needs to be a type that can be ordered.

Let's define the `Ordered[T]` trait. This definition allows values of type `T` to be comparable.

In [2]:
import scala.language.postfixOps

trait Ordered[A] {
    def compare(that: A): Int
    def < (that: A): Boolean = (this compare that) < 0
    def > (that: A): Boolean = (this compare that) > 0
    def <= (that: A): Boolean = (this compare that) <= 0
    def >= (that: A): Boolean = (this compare that) >= 0
    def compareTo(that: A): Int = compare(that)
}

import scala.language.postfixOps
defined trait Ordered

We use the trait `Ordered[T]` as a _type constraint_ in the generic 
type `Set[T]`, so any type `T` is subtype of `Ordered[T]`.

In [7]:
import scala.language.postfixOps

abstract class Set[T <: Ordered[T]] {
    def incl(x: T): Set[T]
    def contains(x: T): Boolean
}

class EmptySet[T <: Ordered[T]] extends Set[T] {
    def contains(x: T): Boolean = false
    def incl(x: T): Set[T] = new NonEmptySet(x, new EmptySet[T], new EmptySet[T])
}

class NonEmptySet[T <: Ordered[T]](e: T, lset: Set[T], rset: Set[T]) extends Set[T] {
    def contains(x: T): Boolean =
        if (x < e) lset contains x // T needs to be comparable
        else if (x > e) rset contains x
        else true
    def incl(x: T): Set[T] =
        if (x < e) new NonEmptySet(e, lset incl x, rset)
        else if (x > e) new NonEmptySet(e, lset, rset incl x)
        else this
}

import scala.language.postfixOps
defined class Set
defined class EmptySet
defined class NonEmptySet

Finally, we define the type `Num` which implement the behaviour of an `Ordered[Num]` type.

In [8]:
case class Num(value: Double) extends Ordered[Num] {
    def compare(that: Num): Int =
    if (this.value < that.value) -1
    else if (this.value > that.value) 1
    else 0
}

defined class Num

In [9]:
val s = new EmptySet[Num].incl(Num(9)).incl(Num(0)).incl(Num(10))
s.contains(Num(0))

s: Set[Num] = ammonite.$sess.cmd7$Helper$NonEmptySet@30cf942d
res9_1: Boolean = true

<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>